In [1]:
import processing as llm
import database as db


# Simulate output of receipt ocr processing

In [38]:
import pandas as pd
# Simulate output from ocr
# ! requesting 140 sequentially triggers the Mistral API limit (5 requests per minute allowed)
# Limit input to receipts 1-14

receipt = pd.read_csv('../data/scanned_rewe_receipts_cleaned.csv', index_col=0).rename(columns={'product_name':'product_abbr'}).iloc[:131]#.query("receipt_id == 'Rewe_1.jpg'")


In [39]:
receipt


,product_abbr,price,receipt_id
0,VOLLKORNTORTILLA,2.58,Rewe_1.jpg
1,TOFU NATUR,4.38,Rewe_1.jpg
2,HAUCHSCHN CURRY,3.18,Rewe_1.jpg
3,HONEYPOMELO PINK,2.99,Rewe_1.jpg
4,DINKEL - CRUNCHY,2.59,Rewe_1.jpg
...,...,...,...
126,KORIANDER,1.98,Rewe_14.jpg
127,THI . REISNUDELN,4.98,Rewe_14.jpg
128,REISPAPIER,3.19,Rewe_14.jpg
129,URKROSTITZ PILS,14.49,Rewe_14.jpg


# Process a receipt with Mistral

In [40]:
llm_processed_receipt = llm.process_receipt(receipt)


Requesting Mixtral for VOLLKORNTORTILLA…
Received response
Parses response successfully, Vollkorn Tortilla
Requesting Mixtral for TOFU NATUR…
Received response
Parses response successfully, Tofu Natur
Requesting Mixtral for HAUCHSCHN CURRY…
Received response
Parses response successfully, Rügenwalder Mühle Veganer Hauchschnitt Typ Hähnchen
Requesting Mixtral for HONEYPOMELO PINK…
Received response
Parses response successfully, Pink Honey Pomelo
Requesting Mixtral for DINKEL - CRUNCHY…
Received response
Parses response successfully, Dinkel Crunchy
Requesting Mixtral for TARTEX SC TOSKA…
Received response
Parses response successfully, Tartex Sc Toskana
Requesting Mixtral for AUFSTRICH PAP . ZU…
Received response
Parses response successfully, Aufstrich Paprika Zucchini
Requesting Mixtral for BROT VITAL GF…
Received response
Parses response successfully, Vollkornbrot Vital (glutenfrei)
Requesting Mixtral for INST . NUDELGER . HU…
Received response
Parses response successfully, Instant Nudel

In [41]:
llm_processed_receipt


,product_abbr,price,receipt_id,productName,categoryMain,categorySub,embedding
0,VOLLKORNTORTILLA,2.58,Rewe_1.jpg,Vollkorn Tortilla,"Brot, Cerealien & Aufstriche",Backwaren,"[-0.045166015625, 0.00507354736328125, 0.03604..."
1,TOFU NATUR,4.38,Rewe_1.jpg,Tofu Natur,Fleisch & Fisch,Fleischalternativen,"[-0.03302001953125, 0.0308990478515625, 0.0634..."
2,HAUCHSCHN CURRY,3.18,Rewe_1.jpg,Rügenwalder Mühle Veganer Hauchschnitt Typ Häh...,Fleisch & Fisch,Fleischalternativen,"[-0.025238037109375, 0.0048675537109375, 0.080..."
3,HONEYPOMELO PINK,2.99,Rewe_1.jpg,Pink Honey Pomelo,Obst & Gemüse,Frisches Obst,"[-0.0192413330078125, 6.914138793945312e-05, 0..."
4,DINKEL - CRUNCHY,2.59,Rewe_1.jpg,Dinkel Crunchy,"Brot, Cerealien & Aufstriche",Müsli & Cerealien,"[0.0006494522094726562, 0.021484375, 0.0436706..."
...,...,...,...,...,...,...,...
126,KORIANDER,1.98,Rewe_14.jpg,Koriander,Obst & Gemüse,Frische Kräuter,"[-0.027587890625, 0.0127716064453125, 0.048339..."
127,THI . REISNUDELN,4.98,Rewe_14.jpg,Thailändische Reisnudeln,Fertiggerichte & Konserven,Gekühlte Nudeln & Teigwaren,"[-0.0210113525390625, -0.008880615234375, 0.05..."
128,REISPAPIER,3.19,Rewe_14.jpg,Reispapier,Kochen & Backen,Backzutaten,"[-0.07257080078125, -0.01154327392578125, 0.06..."
129,URKROSTITZ PILS,14.49,Rewe_14.jpg,Ur-Krostitzer Pilsner,Getränke & Genussmittel,Bier & Biermischgetränke,"[-0.024261474609375, 0.01018524169921875, 0.02..."


# Write to database

In [6]:
# Run setup (only needs to run once)
#db.setup()


Created table receipts
Created table rewe


In [42]:
db.insert_receipt_data(llm_processed_receipt)


Wrote in database.


# Retrieve user history from database

In [43]:
# Retrieve all receipts data
df = db.data()


In [44]:
df


,id_pk,receipt_id,price,product_abbr,product_name,category_main,category_sub,embedding
0,12,Rewe_1.jpg,2.58,VOLLKORNTORTILLA,Vollkorn Tortilla,"Brot, Cerealien & Aufstriche",Backwaren,"[-0.045166016, 0.0050735474, 0.03604126, -0.00..."
1,13,Rewe_1.jpg,4.38,TOFU NATUR,Tofu Natur,Fleisch & Fisch,Fleischalternativen,"[-0.03302002, 0.030899048, 0.06341553, -0.0038..."
2,14,Rewe_1.jpg,3.18,HAUCHSCHN CURRY,Rügenwalder Mühle Veganer Hauchschnitt Typ Häh...,Fleisch & Fisch,Fleischalternativen,"[-0.025238037, 0.0048675537, 0.080444336, -0.0..."
3,15,Rewe_1.jpg,2.99,HONEYPOMELO PINK,Pink Honey Pomelo,Obst & Gemüse,Frisches Obst,"[-0.019241333, 6.914139e-05, 0.05215454, 0.001..."
4,16,Rewe_1.jpg,2.59,DINKEL - CRUNCHY,Dinkel Crunchy,"Brot, Cerealien & Aufstriche",Müsli & Cerealien,"[0.0006494522, 0.021484375, 0.043670654, 0.007..."
...,...,...,...,...,...,...,...,...
126,138,Rewe_14.jpg,1.98,KORIANDER,Koriander,Obst & Gemüse,Frische Kräuter,"[-0.02758789, 0.012771606, 0.048339844, 0.0069..."
127,139,Rewe_14.jpg,4.98,THI . REISNUDELN,Thailändische Reisnudeln,Fertiggerichte & Konserven,Gekühlte Nudeln & Teigwaren,"[-0.021011353, -0.008880615, 0.056030273, 0.01..."
128,140,Rewe_14.jpg,3.19,REISPAPIER,Reispapier,Kochen & Backen,Backzutaten,"[-0.0725708, -0.011543274, 0.06524658, 0.00312..."
129,141,Rewe_14.jpg,14.49,URKROSTITZ PILS,Ur-Krostitzer Pilsner,Getränke & Genussmittel,Bier & Biermischgetränke,"[-0.024261475, 0.010185242, 0.025604248, 0.024..."


# Search in user history

In [45]:
# User search query
query_user_input, n_results, query_table = (['aufstrich für auf dem brot'], 10, 'receipts')

query_embedding = llm.get_embeddings_by_chunks(query_user_input, 1)

query_results = db.search(query_embedding, n_results, query_table)
query_results


,id_pk,receipt_id,price,product_abbr,product_name,category_main,category_sub,embedding
0,18,Rewe_1.jpg,2.38,AUFSTRICH PAP . ZU,Aufstrich Paprika Zucchini,"Brot, Cerealien & Aufstriche",Herzhafte Brotaufstriche,"[-0.050994873, 0.017532349, 0.041900635, 0.005..."
1,43,Rewe_2.jpg,1.35,AUFSTRICH PAP . ZU,Aufstrich Paprika Zucchini,"Brot, Cerealien & Aufstriche",Herzhafte Brotaufstriche,"[-0.050994873, 0.017532349, 0.041900635, 0.005..."
2,108,Rewe_7.jpg,4.05,AUFSTRICH PAP . ZU,Aufstrich Paprika Zucchini,"Brot, Cerealien & Aufstriche",Herzhafte Brotaufstriche,"[-0.05102539, 0.017562866, 0.041900635, 0.0053..."
3,58,Rewe_4.jpg,1.01,KARTOFFELBROT,Kartoffelbrot,"Brot, Cerealien & Aufstriche",Brot,"[-0.04525757, -0.007007599, 0.03930664, -0.004..."
4,44,Rewe_2.jpg,2.49,BROTAUFSTRICH ME,Bio Veganer Brotaufstrich Mandel,"Brot, Cerealien & Aufstriche",Süße Brotaufstriche,"[-0.0021018982, 0.01751709, 0.05053711, -0.012..."
5,56,Rewe_4.jpg,0.63,KRUSTENBROT,Krustenbrot,"Brot, Cerealien & Aufstriche",Brot,"[-0.05328369, 0.0013580322, 0.030578613, -0.00..."
6,28,Rewe_2.jpg,0.76,KRUSTENBROT,Krustenbrot,"Brot, Cerealien & Aufstriche",Brot,"[-0.053222656, 0.0013532639, 0.030593872, -0.0..."
7,62,Rewe_4.jpg,1.07,WELTMEISTERBROT,Weltmeisterbrot,"Brot, Cerealien & Aufstriche",Brot,"[-0.018417358, 0.035369873, 0.032196045, -0.00..."
8,29,Rewe_2.jpg,1.99,WELTM . -MEHRKORN,Weltmeister Mehrkornbrötchen,"Brot, Cerealien & Aufstriche",Brötchen,"[-0.029541016, 0.0362854, 0.044799805, -0.0076..."
9,50,Rewe_3.jpg,6.38,BROT CLASSIC GF,Brot Classic glutenfrei,"Brot, Cerealien & Aufstriche",Brot,"[-0.024673462, 0.01184845, 0.016357422, -0.009..."


# Search in rewe database

In [46]:
# TODO: embeddings of rewe products with category may produce better results
# User search query
query_user_input, n_results, query_table = (['aufstrich für auf dem brot'], 10, 'rewe')

query_embedding = llm.get_embeddings_by_chunks(query_user_input, 1)

query_results = db.search(query_embedding, n_results, query_table)
query_results


,id,name,price,category,embedding
0,11515,Brunch Brotaufstrich Balance 185g,1.89,"Käse, Eier & Molkerei","[-0.01461792, 0.024841309, 0.052368164, 0.0062..."
1,11516,Brunch Brotaufstrich Kräuter 185g,1.89,"Käse, Eier & Molkerei","[-0.022262573, 0.025299072, 0.032806396, -0.00..."
2,857,Glockenbrot Krusti Brötchen,0.69,"Brot, Cerealien & Aufstriche","[-0.03665161, -0.016189575, 0.029067993, -0.01..."
3,858,Glockenbrot Käsebrötchen,0.89,"Brot, Cerealien & Aufstriche","[-0.03427124, -0.0007019043, 0.03161621, -0.00..."
4,1694,ja! Feiner Brotaufstrich Eiersalat 150g,1.29,"Brot, Cerealien & Aufstriche","[-0.011154175, 0.008682251, 0.05218506, -0.018..."
5,863,Glockenbrot Weltmeister Brötchen,0.49,"Brot, Cerealien & Aufstriche","[-0.0137786865, 0.0036010742, 0.02670288, -0.0..."
6,6434,ja! Feiner Brotaufstrich Geflügelsalat 150g,1.29,Fleisch & Fisch,"[-0.015838623, 0.0066375732, 0.053497314, -0.0..."
7,831,Glockenbrot Bauernbrot geschnitten,0.28,"Brot, Cerealien & Aufstriche","[-0.045043945, -0.012191772, 0.02961731, -0.00..."
8,946,Kartoffelbrötchen mit Karotte,1.19,"Brot, Cerealien & Aufstriche","[-0.04537964, 0.008804321, 0.0259552, 0.012672..."
9,859,Glockenbrot Kürbiskernbrötchen,0.39,"Brot, Cerealien & Aufstriche","[-0.037109375, -0.0023021698, 0.025405884, -0...."
